In [1]:
from torchdata import load_file, process_expression, process_json, load_vals, generate_optim_functions
from torchengine import AnalyticalSetSympy, Function, EliminateAnalysis, EliminateAnalysisMergeResiduals
from torchengine import ElimResidual, FunctionSympy
from torchdata import print_formatted_table, fmt
from scipy.optimize import minimize
import sympy as sp
import torch
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})

In [2]:
symb_str_mapping = {}
all_analyses = {}
disciplines = ['pearl_geom','pearl_hydro', 
               'pearl_mass','pearl_prop',
               'pearl_comms','pearl_power', 
               'pearl_solar', 'pearl_battery',
               'pearl_prob']
flattened_output = []
equality_constraints_sympy = []
inequality_constraints_sympy = []
for file_name in disciplines:
    data = load_file(file_name)
    equality_constraints_sympy += [
        process_expression(elt, symb_str_mapping) 
        for elt in data.get('equality_constraints',[])]
    inequality_constraints_sympy += [
        process_expression(elt, symb_str_mapping) 
        for elt in data.get('inequality_constraints',[])]
    objective = data.get('objective',None)
    if objective is not None:
        objective = process_expression(objective, symb_str_mapping)
    functional_sets = data.get('functional_sets',[])
    flattened_output += functional_sets
    analysismap, symb_str_mapping = process_json(
        functional_sets, symb_str_mapping)
    all_analyses[file_name] = analysismap

In [3]:
idxrev = {i: elt for i, elt in 
            enumerate(symb_str_mapping.values())}
indices = {elt: torch.tensor([int(i)]) for i, elt in 
            idxrev.items()}

In [4]:
sets ={}
for file_name, analysismap in all_analyses.items():
        sets[file_name] = {
                idx:AnalyticalSetSympy(analysis, 
                outputvar=outputvar, indices=indices) 
                for idx,(analysis,outputvar) in enumerate(analysismap)
                }
equality_constraints = EliminateAnalysisMergeResiduals(functions=[Function((
        sorted(expr.free_symbols, key=lambda s: s.name),
        sp.lambdify(sorted(expr.free_symbols, key=lambda s: s.name), expr, torch),  
        ), indices=indices) 
        for expr in equality_constraints_sympy])
inequality_constraints = EliminateAnalysisMergeResiduals(functions=[Function((
        sorted(expr.free_symbols, key=lambda s: s.name),
        sp.lambdify(sorted(expr.free_symbols, key=lambda s: s.name), expr, torch),  
        ), indices=indices) 
        for expr in inequality_constraints_sympy])
objective = Function((
        sorted(objective.free_symbols, key=lambda s: s.name),
        sp.lambdify(sorted(objective.free_symbols, key=lambda s: s.name), objective, torch),  
        ), indices=indices)

In [5]:
custom_partition_order = []
flat_sets = {}
for file_name, subsets in sets.items():
    custom_partition_order.append(())
    for elt in subsets.values():
        flatidx = len(flat_sets)
        flat_sets[flatidx] = elt
        custom_partition_order[-1] += (flatidx,)

In [6]:
sequential = {}
for partidx, elt in enumerate(custom_partition_order):
    if len(elt) ==1:
        sequential[partidx] = flat_sets[next(iter(elt))].analysis
    else:
        sequential[partidx] = EliminateAnalysis([flat_sets[idx].analysis for idx in elt], [])

In [7]:
discipline_idxs = [s.analysis.structure[1][0].item() for s in flat_sets.values()]

In [8]:
all_res = EliminateAnalysisMergeResiduals(functions=[s.residual for s in flat_sets.values()])

In [9]:
solve_vars = [idxrev[elt] for elt in discipline_idxs]
#solve_vars += [sp.symbols('D_f')]

In [10]:
np.array(solve_vars)

array([d, V_d, F_B, F_W, K_B, K_G, I, B_M, G_M, C_33, A_33, \omega_0,
       m_platform, m_solar, m_struct, t_d, S_w, P_move, l, L_pt, G_t, G_r,
       S, L_s, P_comms, E_move, E_hotel, E_comms, E_service, P_service,
       E_required, E_recharge, P_recharge, A_solar, C, m_batt],
      dtype=object)

In [11]:
from torchengine import ipoptsolver
from functools import partial

In [12]:
bnds = [(0, None) for _ in solve_vars]

In [13]:
ipsolver = partial(ipoptsolver, bnds_problem=bnds)

In [24]:
MDA = ElimResidual(all_res, solve_vars, indices)

In [25]:
x0 = load_vals('pearl_params', indices=indices)
# numerical_values = {'D_f': 0.3,
#                     'D_s': 0.2,
#                     'D_d': 0.3,
#                     't_f': 0.1,
#                     't_s': 0.2,
#                     't_d': 0.1,
#                     'v':0.1,
#                     'alpha': 0.05,}
# for var, val in numerical_values.items():
#     x0[indices[sp.symbols(var)]] = val

In [26]:
MDA(x0)

tensor([ 1.0511e+01,  2.0000e+00,  5.0000e-02,  1.0000e-01,  1.2500e+00,
         2.0000e+00,  4.5000e-01,  8.9959e-01,  2.0000e+00,  8.8009e+00,
         9.8000e+00,  1.0000e+03,  8.6248e+04,  8.6248e+04,  1.8120e+00,
         5.0000e-01,  1.8375e+00,  7.8540e-01,  8.9241e-02,  6.3727e-02,
         3.0788e+04,  2.9611e+03,  8.8009e+03,  1.3088e+00,  1.0000e+01,
         1.0511e+02,  1.1727e+02,  5.0000e+01,  5.0000e+01,  8.4785e+03,
         7.0000e+02,  2.7000e+03,  1.8239e+01,  1.0000e+00,  7.5000e-01,
         1.0000e+00,  1.2160e+04,  3.0000e+08,  2.2000e+09,  1.3636e-01,
         2.7000e+01,  3.2000e+01,  1.3986e-01,  5.5000e-01,  4.0829e-01,
         5.3000e+00,  8.2001e+03,  6.3780e+06,  6.0000e+05,  2.8308e+06,
        -1.3671e-08,  3.8900e+01,  9.3300e-01,  7.9000e-01,  9.3300e-01,
         8.5000e+07,  1.3500e+02,  1.3807e-23,  1.3884e-12,  3.6000e+03,
         4.3774e+07,  5.0000e+01,  8.6400e+04,  4.3200e+06,  3.6000e+03,
         4.9982e-09,  1.9000e+03,  1.0000e+00,  1.9

In [27]:
all_res(x0).numpy()

array([0, -0, -0, 0, 0, 0, 0, -0, 0, 0, 0, -0, 0, 0, 0, -0, -0, -0, 0, 0,
       0, -0, 0, 0, -0, 0, 0, 0, 0, 0, 0, 0, 0, -0, 0, 0])

In [28]:
disciplineidx = 5
print_formatted_table([x0], 
                      indices, idxrev,  
                      torch.cat(sequential[disciplineidx].structure))

P_move P_comms  t_move P_hotel t_mission t_comms E_AUV G t_service E_move E_hotel E_comms  E_service P_service E_required
1.22e4 1.39e-12 3600   50      8.64e4    3600    1900  1 4.32e4    4.38e7 4.32e6  5.00e-09 1900      0.044     4.81e7    


In [51]:
disciplineidx = 4
print_formatted_table([x0, sequential[disciplineidx](x0)], 
                      indices, idxrev,  
                      torch.cat(sequential[disciplineidx].structure))

d     c      f      e_t theta_t eta_parab D_r R_e    h      E_N  L_a   L_l  L_p   R      T_s k        l     L_pt G_t      G_r     S      L_s      P_comms  
0.015 3.00e8 2.20e9 27  32      0.55      5.3 6.38e6 6.00e5 38.9 0.933 0.79 0.933 8.50e7 135 1.38e-23 0.136 0.14 9.19e-03 8200.06 2.83e6 2.98e-06 -4.56e-07
0.015 3.00e8 2.20e9 27  32      0.55      5.3 6.38e6 6.00e5 38.9 0.933 0.79 0.933 8.50e7 135 1.38e-23 0.136 0.14 9.19e-03 8200.06 2.83e6 1.47e-17 8096.136 


In [45]:
F = FunctionSympy(sp.sympify('(4*A_solar/(pi*(1-alpha)))**0.5'), indices)

In [46]:
F(x0)

tensor([1.1251], dtype=torch.float64)